## 1. Code a function to do median blur operations by your self. Can it be completed in a shorter time complexity?

In [1]:
import cv2
import numpy as np

### 中值滤波 复杂度为O(M * N * k_size * k_size)

In [7]:
def median_blur(img, k_size):
    """
    图像中值滤波
    :param img: 源图像
    :param k_size: 滤波kernel大小，为奇数
    :return:
    """
    if k_size % 2 == 0:
        return img
    # 边界填充
    padding = int(k_size/2)
    img = cv2.copyMakeBorder(img, padding, padding, padding, padding, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    image_info = img.shape
    height = image_info[0]
    width = image_info[1]
    dst = np.zeros(image_info, np.uint8)
    collect_b = np.zeros(k_size * k_size, np.uint8)
    collect_g = np.zeros(k_size * k_size, np.uint8)
    collect_r = np.zeros(k_size * k_size, np.uint8)
    for i in range(padding, height-padding):
        for j in range(padding, width-padding):
            k = 0
            # 记录当前窗口中的像素值
            for m in range(-padding, -padding+k_size):
                for n in range(-padding, -padding+k_size):
                    (b, g, r) = img[i+m, j+n]
                    collect_b[k] = int(b)
                    collect_g[k] = int(g)
                    collect_r[k] = int(r)
                    k += 1
            # 取当前窗口中的像素值中位数作为当前像素值
            b = np.median(collect_b)
            g = np.median(collect_g)
            r = np.median(collect_r)
            dst[i, j] = (b, g, r)
    # 去除填充的边界 返回原图像大小的结果
    return dst[padding:height-padding, padding:width-padding]

In [8]:
image = cv2.imread('face.jpg', 1)
cv2.imshow('face', image)
median_img = median_blur(image, 5)
cv2.imshow('median_face', median_img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

### Huang算法：找到的这个灰度值前面的像素个数（小于该灰度值）和后面的像素个数（大于该灰度值的个数）是相等的）。　　
### O(1)算法：采用两层直方图，高层用于保存高4位（直方图大小为16 * 1），低层保存全位（直方图大小为256 * 1）。每次先累加高层进行范围缩小，再从相应范围累加低层找到中值。这样做牺牲了内存空间。
### 在滤波半径较小的时候，用Huang算法更好，随着滤波半径的增大，O(1)算法会更优势

## 2. Read RANSAC algorithm and write a pseudo code about it.

#### RANSAC（RANdom SAmple Consensus）可以从一组包含“局外点”的观测数据集中，通过迭代方式估计数学模型的参数。它是一种不确定的算法，有一定的概率得出一个合理的结果；为了提高概率必须提高迭代次数。<br><br>RANSAC的基本假设是： <br>（1）数据由“局内点”组成，例如：数据的分布可以用一些模型参数来解释； <br>（2）“局外点”是不能适应该模型的数据；<br>（3）除此之外的数据属于噪声。 <br><br>局外点产生的原因有：噪声的极值；错误的测量方法；对数据的错误假设。 <br><br>RANSAC也做了以下假设：给定一组（通常很小的）局内点，存在一个可以估计模型参数的过程，而该模型能够解释或者适用于局内点。<br><br>RANSAC算法的输入是一组观测数据，一个可以解释或者适应于观测数据的参数化模型，一些可信的参数。 <br><br>RANSAC通过反复选择数据中的一组随机子集来达成目标。被选取的子集被假设为局内点，并用下述方法进行验证： <br>1.首先我们先随机假设一小组局内点为初始值。然后用此局内点拟合一个模型，此模型适应于假设的局内点，所有的未知参数都能从假设的局内点计算得出。 <br>2.用1中得到的模型去测试所有的其它数据，如果某个点适用于估计的模型，认为它也是局内点，将局内点扩充。 <br>3.如果有足够多的点被归类为假设的局内点，那么估计的模型就足够合理。 <br>4.然后，用所有假设的局内点去重新估计模型，因为此模型仅仅是在初始的假设的局内点估计的，后续有扩充后，需要更新。 <br>5.最后，通过估计局内点与模型的错误率来评估模型。 <br><br>整个过程为迭代一次，此过程被重复执行固定的次数，每次产生的模型有两个结局： <br>1、因为局内点太少，还不如上一次的模型，而被舍弃， <br>2、因为比现有的模型更好而被选用。<br><br>RANSAC的优点是它能鲁棒的估计模型参数。例如，它能从包含大量局外点的数据集中估计出高精度的参数。RANSAC的缺点是它计算参数的迭代次数没有上限；如果设置迭代次数的上限，得到的结果可能不是最优的结果，甚至可能得到错误的结果。RANSAC只有一定的概率得到可信的模型，概率与迭代次数成正比。RANSAC的另一个缺点是它要求设置跟问题相关的阀值。 <br><br>RANSAC只能从特定的数据集中估计出一个模型，如果存在两个（或多个）模型，RANSAC不能找到别的模型。

In [ ]:
"""
输入： 
data —— 一组观测数据 
model —— 适应于数据的模型 
n —— 适用于模型的最少数据个数 
k —— 算法的迭代次数 
t —— 用于决定数据是否适应于模型的阀值 
d —— 判定模型是否适用于数据集的数据数目 
输出： 
best_model —— 跟数据最匹配的模型参数（如果没有找到好的模型，返回null） 
best_consensus_set —— 估计出模型的数据点 
best_error —— 跟数据相关的估计出的模型错误
"""
iterations = 0
best_model = None
best_consensus_set = None
best_error = 无穷大
while ( iterations < k )
    maybe_inliers = 从数据集中随机选择n个点
    maybe_model = 适合于maybe_inliers的模型参数
    consensus_set = maybe_inliers

    for ( 每个数据集中不属于maybe_inliers的点 ）
        if ( 如果点适合于maybe_model，且错误小于t ）
            将点添加到consensus_set
    if （ consensus_set中的元素数目大于d ）# 已经找到了好的模型，现在测试该模型到底有多好
        better_model = 适合于consensus_set中所有点的模型参数
        this_error = better_model
        if ( this_error < best_error )
            # 我们发现了比以前好的模型，保存该模型直到更好的模型出现
            best_model =  better_model
            best_consensus_set = consensus_set
            best_error =  this_error